# Preprocessing

In [1]:
import pandas

df = pandas.read_csv('s&p500.csv')
df = df.set_index('Date')
df

,A,AAL,AAPL,ABBV,ABT,ACGL,ACN,ADBE,ADI,ADM,...,WTW,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2014-10-27 00:00:00+00:00,35.544605,37.720184,23.331984,39.992973,34.910320,18.703333,66.987968,67.029999,37.750469,33.726379,...,92.664764,22.832506,157.285110,23.965683,60.671715,30.936274,41.624443,94.137535,69.650002,34.025173
2014-10-28 00:00:00+00:00,35.985706,38.032078,23.693810,40.283882,35.125217,18.756666,67.369476,68.370003,38.050076,34.535152,...,93.792351,22.899595,158.638138,24.148184,61.571686,32.226025,41.965137,96.935326,71.330002,34.434544
2014-10-29 00:00:00+00:00,35.880356,37.748539,23.826988,39.655781,35.323566,18.743334,67.437271,67.930000,38.309196,34.633411,...,90.567459,22.385298,159.220413,23.980284,61.241489,31.545813,41.606518,96.980148,71.269997,34.071690
2014-10-30 00:00:00+00:00,35.761864,38.184391,23.747078,40.449177,35.943420,18.723333,68.344368,68.570000,38.058174,35.207859,...,90.341949,22.696587,160.693359,24.469378,61.150833,31.634153,41.827667,97.993439,71.169998,34.722980
2014-10-31 00:00:00+00:00,36.393875,39.179272,23.973497,41.956593,36.026051,18.773333,68.768234,70.120003,40.179714,35.525322,...,91.401863,22.913132,162.714310,24.432877,62.614044,32.120022,42.933434,99.751015,73.750000,34.574116
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-10-18 00:00:00+00:00,138.830002,13.090000,235.000000,188.860001,119.389999,108.529999,376.859985,494.899994,228.330002,56.400002,...,290.029999,33.060001,99.440002,64.070000,120.010002,134.320007,134.270004,106.309998,375.450012,193.279999
2024-10-21 00:00:00+00:00,136.759995,12.970000,236.479996,186.539993,116.989998,108.500000,376.029999,497.709991,225.669998,56.130001,...,290.500000,32.290001,98.370003,63.430000,120.080002,132.520004,134.000000,105.199997,372.589996,189.449997
2024-10-22 00:00:00+00:00,133.460007,12.960000,235.860001,188.759995,116.120003,107.790001,371.790009,493.109985,224.619995,55.689999,...,291.739990,31.650000,99.400002,63.680000,120.699997,131.369995,133.360001,104.260002,372.000000,189.509995


In [64]:
import torch

def preprocess(df):
    data = torch.from_numpy(df.to_numpy()).to(torch.float32)
    # calculate daily return ratio
    for d in range(1, data.size(0)):
        data[d] = (data[d] - data[d-1]) / data[d-1]
    # skip the first day which cannot calculate daily return ratio
    # and round data size to nearest multiple of batch_size
    data = data[1:2511]
    # split into batches
    data = data.view(251, 10, 493)
    return data

train_data = preprocess(df)
print(train_data.size())
print(train_data[0])

torch.Size([251, 10, 493])
tensor([[-1.0289e+00, -1.0272e+00, -1.0447e+00,  ..., -1.0107e+00,
         -1.0146e+00, -1.0303e+00],
        [-2.7341e+03, -4.3306e+03, -1.4113e+03,  ..., -3.1955e+03,
         -2.8721e+03, -2.6713e+03],
        [-9.9963e-01, -9.9977e-01, -9.9929e-01,  ..., -9.9969e-01,
         -9.9965e-01, -9.9963e-01],
        ...,
        [ 1.1249e+01,  1.2341e+01,  6.7963e+00,  ...,  3.1608e+01,
          2.3360e+01,  1.0947e+01],
        [-1.0798e+00, -1.0735e+00, -1.1204e+00,  ..., -1.0306e+00,
         -1.0408e+00, -1.0816e+00],
        [ 7.8027e+00,  8.5954e+00,  4.4241e+00,  ...,  2.3155e+01,
          1.6659e+01,  7.4500e+00]])


# Model

In [111]:
import torch.nn as nn
import math

# https://towardsdatascience.com/build-your-own-transformer-from-scratch-using-pytorch-84c850470dcb
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        assert d_model % num_heads == 0, "d_model must be divisible by num_heads"
        
        self.d_model = d_model
        self.num_heads = num_heads
        self.d_k = d_model // num_heads
        
        self.W_q = nn.Linear(d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)
        self.W_o = nn.Linear(d_model, d_model)
        
    def scaled_dot_product_attention(self, Q, K, V, mask=None):
        attn_scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)
        if mask is not None:
            attn_scores = attn_scores.masked_fill(mask == 0, -1e9)
        attn_probs = torch.softmax(attn_scores, dim=-1)
        output = torch.matmul(attn_probs, V)
        return output
        
    def split_heads(self, x):
        batch_size, seq_length, d_model = x.size()
        return x.view(batch_size, seq_length, self.num_heads, self.d_k).transpose(1, 2)
        
    def combine_heads(self, x):
        batch_size, _, seq_length, d_k = x.size()
        return x.transpose(1, 2).contiguous().view(batch_size, seq_length, self.d_model)
        
    def forward(self, Q, K, V, mask=None):
        Q = self.split_heads(self.W_q(Q))
        K = self.split_heads(self.W_k(K))
        V = self.split_heads(self.W_v(V))
        
        attn_output = self.scaled_dot_product_attention(Q, K, V, mask)
        output = self.W_o(self.combine_heads(attn_output))
        return output

In [112]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class FeedForward(nn.Module):
    def __init__(self, hidden_size, expand_ratio, dropout):
        super(FeedForward, self).__init__()
        self.linear = nn.Linear(hidden_size, hidden_size * expand_ratio)
        self.linear2 = nn.Linear(hidden_size * expand_ratio, hidden_size)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=dropout)
    
    def forward(self, x):
        x = self.linear(x)
        x = self.relu(x)
        x = self.linear2(x)
        x = self.dropout(x)
        return x

class Attention(nn.Module):
    def __init__(self, d_model, num_heads, expand_ratio, dropout):
        super().__init__()
        # self.mha = nn.MultiheadAttention(embed_dim=d_model, num_heads=num_heads, batch_first=True)
        self.mha = MultiHeadAttention(d_model, num_heads)
        self.ln1 = nn.LayerNorm(d_model)
        self.ln2 = nn.LayerNorm(d_model)
        self.ffn = FeedForward(hidden_size=d_model, expand_ratio=expand_ratio, dropout=dropout)

    def forward(self, x, attn_mask=None):
        x = torch.randn(x.size()).to(torch.float32)
        # x1, _ = self.mha(x, x, x, attn_mask=attn_mask)
        x1 = self.mha(x, x, x, mask=attn_mask)
        x2 = self.ln1(x + x1)
        return self.ln2(self.ffn(x2) + x2)

class SpatialTemporalAttention(nn.Module):
    def __init__(self, d_model=64, num_heads=2, expand_ratio=4, dropout=0.1, T=9, N=493):
        super().__init__()
        self.d_model = d_model
        self.num_heads = num_heads
        self.input_proj = nn.Linear(1, d_model)
        self.time_embedding = nn.Embedding(T, d_model)
        self.stock_embedding = nn.Embedding(N, d_model)
        self.spatial_attn = Attention(d_model, num_heads, expand_ratio, dropout)
        self.temporal_attn = Attention(d_model, num_heads, expand_ratio, dropout)
        self.output_proj = nn.Linear(d_model, 1)
    
    def forward(self, x):
        T, N = x.size()
        # nn.MultiheadAttention returns all NaNs if stock_embedding is not added for some reason
        x = self.input_proj(x.view(T, N, 1)) + self.stock_embedding(torch.arange(N).unsqueeze(0).expand(T, N))
        # IDEA: Each spatial head takes in a different type of correlation matrix.
        # Like one takes in positive pearson's coefficnet and the other takes in negative
        x = self.spatial_attn(x).view(N, T, self.d_model)
        x = self.time_embedding(torch.arange(T).unsqueeze(0).expand(N, T)) + x
        # temporal_causal_mask = torch.tril(torch.ones((T, T), dtype=torch.bool)).expand(N * self.num_heads, T, T).to(x.device)
        temporal_causal_mask = torch.tril(torch.ones((T, T), dtype=torch.bool)).to(x.device)
        x = self.temporal_attn(x, attn_mask=temporal_causal_mask)
        return self.output_proj(x).view(T, N)


# Training

In [115]:
# device = torch.device('cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu')
device = torch.device('cpu')

model = SpatialTemporalAttention().to(device)
train_data = train_data.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
loss_fn = nn.MSELoss()

model.train()

num_epochs = 1

for epoch in range(num_epochs):
    optimizer.zero_grad()
    for batch in train_data:
        out = model(batch[:-1, :])
        print(out.size())
        print(out)
        print(batch[1:, :].size())
        print(batch[1:, :])
        loss = loss_fn(out, batch[1:, :])
        loss.backward()
        print(loss)
        optimizer.step()

torch.Size([9, 493])
tensor([[ 0.6499, -0.4272, -0.2525,  ...,  0.3072,  0.7079,  0.6653],
        [ 0.7881, -0.4980,  0.3192,  ...,  0.2632,  0.0741, -0.0277],
        [-0.3400,  0.4011, -0.4973,  ..., -0.5914, -0.4510, -0.5213],
        ...,
        [-0.0419, -0.0248,  0.3322,  ...,  0.2296, -0.3784, -0.4371],
        [-0.0999,  0.3221,  0.9144,  ..., -0.8997, -0.3410,  0.5195],
        [ 0.3737, -0.9555,  1.0628,  ...,  0.1606,  0.8244, -0.4063]],
       grad_fn=<ViewBackward0>)
torch.Size([9, 493])
tensor([[-2.7341e+03, -4.3306e+03, -1.4113e+03,  ..., -3.1955e+03,
         -2.8721e+03, -2.6713e+03],
        [-9.9963e-01, -9.9977e-01, -9.9929e-01,  ..., -9.9969e-01,
         -9.9965e-01, -9.9963e-01],
        [ 3.6864e+01,  3.9519e+01,  2.4368e+01,  ...,  1.0287e+02,
          7.5597e+01,  3.5017e+01],
        ...,
        [ 1.1249e+01,  1.2341e+01,  6.7963e+00,  ...,  3.1608e+01,
          2.3360e+01,  1.0947e+01],
        [-1.0798e+00, -1.0735e+00, -1.1204e+00,  ..., -1.0306e+00,


KeyboardInterrupt: 